# Load data

In [1]:
import sage
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Load data
df = sage.datasets.bank()

# Convert binary features to 0/1
binary_cols = ['Default', 'Housing', 'Loan']
for col in binary_cols:
    df[col] = (df[col] == 'yes').astype(float)
    
# Convert education to numerical
df['Education'].replace(
    {'unknown': 0, 'primary': 1, 'secondary': 2, 'tertiary': 3},
    inplace=True)

# Convert month to numerical
df['Month'].replace(
    {'jan': 0, 'feb': 1, 'mar': 2, 'apr': 3, 'may': 4, 'jun': 5,
     'jul': 6, 'aug': 7, 'sep': 8, 'oct': 9, 'nov': 10, 'dec': 11},
    inplace=True)

# Convert marital to one-hot
for value in np.unique(df['Marital'].values):
    df['Marital-{}'.format(value)] = (df['Marital'] == value).astype(float)
df.drop(columns='Marital', inplace=True)

# Convert contact to one-hot
for value in np.unique(df['Contact'].values):
    df['Contact-{}'.format(value)] = (df['Contact'] == value).astype(float)
df.drop(columns='Contact', inplace=True)

# Convert prev outcome to one-hot
for value in np.unique(df['Prev Outcome'].values):
    df['Prev Outcome-{}'.format(value)] = (df['Prev Outcome'] == value).astype(float)
df.drop(columns='Prev Outcome', inplace=True)

# Convert job to one-hot
for value in np.unique(df['Job'].values):
    df['Job-{}'.format(value)] = (df['Job'] == value).astype(float)
df.drop(columns='Job', inplace=True)

# Split into X, Y
values = df.values.astype(float)
X_cols = np.array(df.columns) != 'Success'
X, Y = values[:, X_cols], values[:, ~X_cols]

# Get feature names, groups
feature_names = np.array(df.columns)[X_cols]
prefixes = np.array([name.split('-')[0] for name in feature_names])
groups = []
group_names = []
for prefix in np.unique(prefixes):
    groups.append(np.where(prefixes == prefix)[0])
    group_names.append(prefix)

# Train/val/test split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1, random_state=123)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.1, random_state=123)

In [3]:
# Standardize continuous columns
feature_names = list(feature_names)
num_features = len(feature_names)
continuous_cols = ['Age', 'Balance', 'Day', 'Duration', 'Campaign',
                   'Month', 'Prev Days', 'Prev Contacts']
continuous_inds = [feature_names.index(col) for col in continuous_cols]
ss = StandardScaler()
ss.fit(X_train[:, continuous_inds])
X_train[:, continuous_inds] = ss.transform(X_train[:, continuous_inds])
X_val[:, continuous_inds] = ss.transform(X_val[:, continuous_inds])
X_test[:, continuous_inds] = ss.transform(X_test[:, continuous_inds])

# Set up imputer

In [4]:
import torch
import fastshap_torch
from fastshap_torch import Surrogate

In [5]:
device = torch.device('cuda', 5)
surrogate = torch.load('../models/bank_surrogate.pt').eval().to(device)
num_features = X_train.shape[1]
surr = Surrogate(surrogate, num_features, groups)

# FastSHAP

In [6]:
import torch
import torch.nn as nn
from fastshap_torch import FastSHAP

# Test samples number

In [7]:
for n_samples in (1, 4, 16, 32, 48, 64, 96):
    # Set up explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2 * len(groups))).to(device)

    # Set up FastSHAP wrapper
    fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

    # Train
    fastshap.train(
        X_train,
        X_val[:500],
        batch_size=32,
        num_samples=n_samples,
        max_epochs=200,
        eff_lambda=0,
        paired_sampling=False,
        validation_samples=128,
        validation_seed=123,
        verbose=False)

    # Print performance
    print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

    # Save model
    modifier = 'samples={}'.format(n_samples)
    explainer.cpu()
    torch.save(explainer, '../models/bank_explainer {} nopenalty.pt'.format(modifier))

Best val loss = 0.00311171
Best val loss = 0.00047078
Best val loss = 0.00005620
Best val loss = 0.00005627
Best val loss = 0.00005515
Best val loss = 0.00005536
Best val loss = 0.00005495


In [8]:
for n_samples in (4, 16, 32, 48, 64, 96):
    # Set up explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2 * len(groups))).to(device)

    # Set up FastSHAP wrapper
    fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

    # Train
    fastshap.train(
        X_train,
        X_val[:500],
        batch_size=32,
        num_samples=n_samples,
        max_epochs=200,
        eff_lambda=0,
        paired_sampling=True,
        validation_samples=128,
        validation_seed=123,
        verbose=False)

    # Print performance
    print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

    # Save model
    modifier = 'paired_samples={}'.format(n_samples)
    explainer.cpu()
    torch.save(explainer, '../models/bank_explainer {} nopenalty.pt'.format(modifier))

Best val loss = 0.00046258
Best val loss = 0.00005534
Best val loss = 0.00005523
Best val loss = 0.00005464
Best val loss = 0.00005514
Best val loss = 0.00005465


# Test other parameters

In [11]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * len(groups))).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train,
    X_val[:500],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nopenalty'
explainer.cpu()
torch.save(explainer, '../models/bank_explainer {}.pt'.format(modifier))

In [12]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * len(groups))).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization=None, link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train,
    X_val[:500],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0.1,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nonormalization'
explainer.cpu()
torch.save(explainer, '../models/bank_explainer {}.pt'.format(modifier))

In [13]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * len(groups))).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization=None, link=nn.Softmax(dim=-1))

# Train
fastshap.train(
    X_train,
    X_val[:500],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nopenalty nonormalization'
explainer.cpu()
torch.save(explainer, '../models/bank_explainer {}.pt'.format(modifier))